## Linear Elasticity for Cantilever Beam

In [1]:
using Gridap, Gridap.CellData, SparseArrays, BenchmarkTools
using NearestNeighbors, Printf  # make sure this is at the top of your file
using LaTeXStrings

In [2]:
using Gridap
using GridapGmsh
using LinearAlgebra
using SparseArrays

# 1. Define geometry and mesh (using Gmsh for mesh generation)
L = 20  # Length of heat sink base
W = 20  # Width and Height
domain = (0, L, 0, W, 0, W)
partition = (20*2, 20*2, 20*2)
model = CartesianDiscreteModel(domain, partition)

Bottom = [1, 2, 5, 6, 9, 11, 17, 18, 23]
Top = [3,4,7,8,10,12,19,20,24]
diff_ = setdiff(1:26, Bottom)

# === Tags for boundary conditions ===
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "bottom", Bottom) # example indices for corners/edges
add_tag_from_tags!(labels, "top", Top) # Top


degree = 2
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, degree)
V = TestFESpace(model, reffe; dirichlet_tags=["bottom"])
U = TrialFESpace(V, [VectorValue(0.0, 0.0)]) # Fixed bottom boundary



# 2. Material properties
E = 30e9  # Young's modulus (Pa)
ν = 0.2   # Poisson's ratio
ρ = 2400  # Density (kg/m³)
λ = E * ν / ((1 + ν) * (1 - 2 * ν))  # Lamé parameters
μ = E / (2 * (1 + ν))
C = (λ, μ)  # Constitutive tensor components

# 3. CONWEP blast pressure function
function conwep_pressure(t, R=1.0, W=25.0)
    # Simplified CONWEP parameters (based on scaled distance Z = R / W^(1/3))
    Z = R / W^(1/3)
    # Empirical values for peak pressure and duration (approximated from UFC 3-340-02)[](https://www.masonryandhardscapes.org/resource/tek-14-21a/)
    Pr = 1.8e6  # Peak reflected pressure (Pa) for Z ≈ 0.4 m/kg^(1/3)
    td = 0.003  # Positive phase duration (s)
    if t >= 0 && t <= td
        P = Pr * (1 - t / td) * exp(-t / td)
    else
        P = 0.0
    end
    return P
end

# 4. Weak form and matrices
dΩ = Measure(model, degree)
a(u, v) = ∫( λ * div(u) * div(v) + 2 * μ * (symmetric_gradient(u) ⋅ symmetric_gradient(v)) ) * dΩ
m(u, v) = ∫( ρ * u ⋅ v ) * dΩ

# External load (blast pressure applied on top surface)
dΓ = Measure(model, degree, tag="top")
f(t) = v -> ∫( VectorValue(0.0, -conwep_pressure(t)) ⋅ v ) * dΓ

# Assemble stiffness and mass matrices
K = assemble_matrix(a, U, V)
M = assemble_matrix(m, U, V)

# 5. Time-stepping (Newmark-beta method)
Δt = 0.0001  # Time step (s)
T = 0.01      # Total simulation time (s)
β = 0.25      # Newmark parameters
γ = 0.5
u0 = zeros(num_free_dofs(U))  # Initial displacement
v0 = zeros(num_free_dofs(U))  # Initial velocity
a0 = zeros(num_free_dofs(U))  # Initial acceleration

# Precompute effective stiffness matrix
K_eff = M + β * Δt^2 * K

# Time loop
u = u0
v = v0
a = a0
nt = Int(T / Δt)
displacements = [u]
for n in 1:nt
    t = n * Δt
    F = assemble_vector(f(t), V)  # Blast load at time t
    # Newmark-beta update
    rhs = F - K * u - M * (a + Δt * v + (0.5 - β) * Δt^2 * a)
    a_new = K_eff \ rhs
    v_new = v + Δt * ((1 - γ) * a + γ * a_new)
    u_new = u + Δt * v + Δt^2 * ((0.5 - β) * a + β * a_new)
    u, v, a = u_new, v_new, a_new
    push!(displacements, u)
end

# 6. Post-processing (e.g., displacement at a point)
write_vtk(model, "results", cellfields=["displacement"=>displacements[end]])

MethodError: MethodError: no method matching CellQuadrature(::CartesianDiscreteModel{3, Float64, typeof(identity)}, ::Int64)

Closest candidates are:
  CellQuadrature(!Matched::Gridap.Geometry.AppendedTriangulation, ::Any, !Matched::Any; data_domain_style, integration_domain_style, kwargs...)
   @ Gridap C:\Users\snv22002\.julia\packages\Gridap\lRBm2\src\CellData\CellQuadratures.jl:61
  CellQuadrature(!Matched::Triangulation, ::Any, !Matched::DomainStyle)
   @ Gridap deprecated.jl:103
  CellQuadrature(!Matched::Gridap.Geometry.AppendedTriangulation, ::Any, !Matched::Any, !Matched::DomainStyle)
   @ Gridap deprecated.jl:103
  ...


In [3]:
using Gridap
using GridapGmsh
using LinearAlgebra
using SparseArrays

# 1. Define CONWEP blast pressure function
function conwep_pressure(t, R=1.0, W=25.0)
    Z = R / W^(1/3)  # Scaled distance
    Pr = 1.8e6       # Peak reflected pressure (Pa), approximated
    td = 0.003       # Positive phase duration (s), approximated
    if 0 <= t <= td
        P = Pr * (1 - t / td) * exp(-t / td)
    else
        P = 0.0
    end
    return P
end

# 2. Geometry and mesh
L = 10  # Length of heat sink base
W = 10  # Width and Height
domain = (0, L, 0, W, 0, W)
partition = (10, 10, 10)
model = CartesianDiscreteModel(domain, partition)

Bottom = [1, 2, 5, 6, 9, 11, 17, 18, 23]
Top = [3,4,7,8,10,12,19,20,24]
diff_ = setdiff(1:26, Bottom)

# === Tags for boundary conditions ===
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "bottom", Bottom) # example indices for corners/edges
add_tag_from_tags!(labels, "top", Top) # Top


order = 2
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, order)
V0 = FESpace(
    model,
    reffe,
    conformity=:H1,
    dirichlet_tags=["bottom"])  # Fixed bottom boundary
U = TransientTrialFESpace(V0, t -> VectorValue(0.0, 0.0))

# 3. Material properties
E = 30e9  # Young's modulus (Pa)
ν = 0.2   # Poisson's ratio
ρ = 2400  # Density (kg/m³)
λ = E * ν / ((1 + ν) * (1 - 2 * ν))  # Lamé parameters
μ = E / (2 * (1 + ν))

# 4. Weak form
Ω = Triangulation(model)
degree = 2 * order
dΩ = Measure(Ω, degree)
dΓ = Measure(model, degree, tag="top")  # Blast load on top surface

m(t, utt, v) = ∫(ρ * (utt ⋅ v))dΩ
c(t, ut, v) = ∫(0.0 * (ut ⋅ v))dΩ  # No damping for simplicity
a(t, u, v) = ∫(λ * div(u) * div(v) + 2 * μ * (symmetric_gradient(u) ⊙ symmetric_gradient(v)))dΩ
b(t, v) = ∫(VectorValue(0.0, -conwep_pressure(t)) ⋅ v)dΓ  # Blast load as traction

# 5. Transient FE operator
res(t, u, v) = m(t, ∂tt(u), v) + c(t, ∂t(u), v) + a(t, u, v) - b(t, v)
jac(t, u, du, v) = a(t, du, v)
jac_t(t, u, dut, v) = c(t, dut, v)
jac_tt(t, u, dutt, v) = m(t, dutt, v)

op = TransientFEOperator(res, jac, jac_t, jac_tt, U, V0)

# 6. Initial conditions
U0 = U(0.0)
uh0 = interpolate_everywhere(VectorValue(0.0, 0.0), U0)  # Zero initial displacement
vh0 = interpolate_everywhere(VectorValue(0.0, 0.0), U0)  # Zero initial velocity
ah0 = interpolate_everywhere(VectorValue(0.0, 0.0), U0)  # Zero initial acceleration

# 7. Time integration (Newmark-beta)
t0 = 0.0
tF = 0.01
dt = 0.0001
γ = 0.5
β = 0.25
ls = LUSolver()
ode_solver = Newmark(ls, dt, γ, β)

# 8. Solve
sol_t = solve(ode_solver, op, (uh0, vh0, ah0), t0, tF)

# 9. Post-processing
_t_n = t0
for (uh_tn, tn) in sol_t
    _t_n += dt
    write_vtk(model, "results_$(lpad(Int(tn/dt),4,'0'))", cellfields=["displacement"=>uh_tn])
end


MethodError: MethodError: no method matching CellQuadrature(::CartesianDiscreteModel{3, Float64, typeof(identity)}, ::Int64; tag::String)

Closest candidates are:
  CellQuadrature(!Matched::Gridap.Geometry.AppendedTriangulation, ::Any, !Matched::Any; data_domain_style, integration_domain_style, kwargs...)
   @ Gridap C:\Users\snv22002\.julia\packages\Gridap\lRBm2\src\CellData\CellQuadratures.jl:61
  CellQuadrature(!Matched::Triangulation, ::Any, !Matched::DomainStyle) got unsupported keyword argument "tag"
   @ Gridap deprecated.jl:103
  CellQuadrature(!Matched::Gridap.Geometry.AppendedTriangulation, ::Any, !Matched::Any, !Matched::DomainStyle) got unsupported keyword argument "tag"
   @ Gridap deprecated.jl:103
  ...
